# Purpose

2021-10-15: Use this notebook to test & check logs for running new clustering script. Use this as a way to document different options for running the script with hydra.

Focus:
- clustering algos at **subreddit-level.**


# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly
import seaborn as sns

import mlflow
import hydra

import subclu
from subclu.eda.aggregates import compare_raw_v_weighted_language
from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl, 
    get_color_dict, base_colors_for_manual_labels,
    check_colors_used,
)
from subclu.data.data_loaders import LoadPosts, LoadSubreddits, create_sub_level_aggregates


# ===
# imports specific to this notebook
from collections import Counter
# import umap
# import openTSNE
# from openTSNE import TSNE

# import hdbscan

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize  # if we normalize the data, euclidean distance is approx of cosine

from sklearn.cluster import KMeans, DBSCAN, OPTICS, AgglomerativeClustering

print_lib_versions([hydra, np, pd, plotly, sklearn, sns, subclu])

python		v 3.7.10
===
hydra		v: 1.1.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
sklearn		v: 0.24.1
seaborn		v: 0.11.1
subclu		v: 0.4.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas')

,experiment_id,name,artifact_location,lifecycle_stage
0,0,Default,./mlruns/0,active
1,1,fse_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/1,active
2,2,fse_vectorize_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/2,active
3,3,subreddit_description_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/3,active
4,4,fse_vectorize_v1.1,gs://i18n-subreddit-clustering/mlflow/mlruns/4,active
5,5,use_multilingual_v0.1_test,gs://i18n-subreddit-clustering/mlflow/mlruns/5,active
6,6,use_multilingual_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/6,active
7,7,use_multilingual_v1_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/7,active
8,8,use_multilingual_v1_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/8,active
9,9,v0.3.2_use_multi_inference_test,gs://i18n-subreddit-clustering/mlflow/mlruns/9,active


## Get experiment IDs to use for clustering

There are two runs that completed and they both have the same parameters, so we should be able to use either one. For now, let's select:<br>
`0591fdae9b7d4da7ae3839767b8aab66`

In [6]:
%%time

df_mlf = mlf.search_all_runs(experiment_ids=[16])
df_mlf.shape

CPU times: user 49.6 ms, sys: 13.7 ms, total: 63.2 ms
Wall time: 62.4 ms


(13, 86)

In [7]:
mask_finished = df_mlf['status'] == 'FINISHED'
mask_df_similarity_complete = ~df_mlf['metrics.df_sub_level_agg_a_post_only_similarity-rows'].isnull()

df_mlf_clustering_candidates = df_mlf[mask_finished & mask_df_similarity_complete]
df_mlf_clustering_candidates.shape

(2, 86)

In [8]:
cols_with_multiple_vals = df_mlf_clustering_candidates.columns[df_mlf_clustering_candidates.nunique(dropna=False) > 1]

df_mlf_clustering_candidates[cols_with_multiple_vals]

,run_id,artifact_uri,start_time,end_time,metrics.memory_used,metrics.vectorizing_time_minutes,metrics.memory_free,metrics.memory_used_percent,params.memory_used,params.f_log_file,params.memory_free,params.memory_used_percent,params.run_name,tags.mlflow.runName
0,cbb12818e82345dda96928bfdab8b16b,gs://i18n-subreddit-clustering/mlflow/mlruns/16/cbb12818e82345dda96928bfdab8b16b/artifacts,2021-10-12 10:46:05.235000+00:00,2021-10-12 16:41:33.492000+00:00,702999.0,355.468028,3465918.0,0.181436,278514,logs/AggregateEmbeddings/2021-10-12_10-46-05_agg_full_lc_false_pd-2021-10-12_104604.log,3465918,0.07188136995649137,agg_full_lc_false_pd-2021-10-12_104604,agg_full_lc_false_pd-2021-10-12_104604
1,0591fdae9b7d4da7ae3839767b8aab66,gs://i18n-subreddit-clustering/mlflow/mlruns/16/0591fdae9b7d4da7ae3839767b8aab66/artifacts,2021-10-12 10:27:33.324000+00:00,2021-10-12 16:40:41.501000+00:00,703208.0,373.134208,3681161.0,0.181490,64759,logs/AggregateEmbeddings/2021-10-12_10-27-33_agg_full_lc_false-2021-10-12_102732.log,3681161,0.016713578624458464,agg_full_lc_false-2021-10-12_102732,agg_full_lc_false-2021-10-12_102732


# Inspect config for clustering job

This config should include:
- data to load for clustering
- parameters for clustering algo
- hydra overrides to run jobs in parallel

In [9]:
test_experiment = 'v0.4.0_use_multi_clustering_test'

cfg_cluster_test_v040 = LoadHydraConfig(
    config_name='clustering_v0.4.0_base',
    config_path="../config",
    overrides=[
        f"mlflow_experiment_name={test_experiment}"
#         f"data_text_and_metadata=top_subreddits_2021_07_16",
#         f"data_embeddings_to_cluster=top_subs-2021_07_16-use_multi_lower_case_false_00",
    ],
)

print([k for k in cfg_cluster_test_v040.config_dict.keys()])

['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline']


In [10]:
# data with embeddings
cfg_cluster_test_v040.config_dict['data_embeddings_to_cluster']

{'run_uuid': '0591fdae9b7d4da7ae3839767b8aab66',
 'l_ix_sub': ['subreddit_name', 'subreddit_id'],
 'l_ix_post': ['subreddit_name', 'post_id'],
 'df_post_level_agg_b_post_and_comments': None,
 'df_post_level_agg_c_post_comments_sub_desc': 'df_post_level_agg_c_post_comments_sub_desc',
 'df_sub_level_agg_a_post_only': 'df_sub_level_agg_a_post_only',
 'df_sub_level_agg_a_post_only_similarity': 'df_sub_level_agg_a_post_only_similarity',
 'df_sub_level_agg_a_post_only_similarity_pair': 'df_sub_level_agg_a_post_only_similarity_pair',
 'df_sub_level_agg_a_post_only_similarity_top_pair': 'df_sub_level_agg_a_post_only_similarity_top_pair',
 'df_sub_level_agg_b_post_and_comments': None,
 'df_sub_level_agg_b_post_and_comments_similarity': None,
 'df_sub_level_agg_b_post_and_comments_similarity_pair': None,
 'df_sub_level_agg_c_post_comments_and_sub_desc': 'df_sub_level_agg_c_post_comments_and_sub_desc',
 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity': 'df_sub_level_agg_c_post_comments_

In [11]:
# clustering algo
cfg_cluster_test_v040.config_dict['clustering_algo']

{'model_name': 'AgglomerativeClustering',
 'model_kwargs': {'n_clusters': 100,
  'affinity': 'euclidean',
  'linkage': 'ward',
  'connectivity': False}}

# Run commmand line fxn

The clustering fxn is in `subclu.models.conlustering.py`

Notes:
- We need to use the `-m` flag to run as a submodule (and allow relative imports)
- When using the `-m` flag, REMOVE the `.py` ending of the file!
- In the command line for hydra we can override w/o having to use the `+` sign.
    - https://hydra.cc/docs/tutorials/basic/your_first_app/config_file

## Check & set paths

In [12]:
test_experiment

'v0.4.0_use_multi_clustering_test'

In [13]:
os.getcwd()

'/home/jupyter/subreddit_clustering_i18n/notebooks/v0.4.0'

In [14]:
path_djb_repo = '/home/david.bermejo/repos/subreddit_clustering_i18n/' 
path_djb_models = '/home/david.bermejo/repos/subreddit_clustering_i18n/subclu/models' 
file_clustering_py = 'subclu.models.clustering'

In [15]:
# !ls

In [16]:
# !cd $path_djb_repo && ls

In [17]:
# !cd $path_djb_models && ls

## Run clustering from CLI

In [30]:
!cd $path_djb_repo && python -m $file_clustering_py mlflow_experiment_name=$test_experiment n_sample_embedding_rows=4000

CFG keys: dict_keys(['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline'])
`2021-10-25 19:32:46,278` | `INFO` | `Define cluster class...`
`2021-10-25 19:32:46,988` | `INFO` | `== Start run_aggregation() method ==`
`2021-10-25 19:32:46,989` | `INFO` | `MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db`
`2021-10-25 19:32:47,131` | `INFO` | `=== START CLUSTERING - Process ID 81470`
`2021-10-25 19:32:47,230` | `INFO` | `host_name: djb-100-2021-04-28-djb-eda-german-subs`
`2021-10-25 19:32:47,230` | `INFO` | `cpu_count: 96`
`2021-10-25 19:32:47,302` | `INFO` | `RAM stats:
{'memory_used_percent': '0.89%', 'memory_total': '628,888', 'memory_used': '5,627', 'memory_free': '620,388'}`
`2021-10-25 19:32:47,396` | `INFO` | `Using hydra's path`
  Current working di

In [87]:
# os.getcwd()

# Test creating pipeline & removing steps

how easy is it to start with a full pipeline and remove steps v. start with base pipeline and append steps? appending steps could be tricky b/c we'd have to add them in the correct sequence.

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer


pipe_full = Pipeline([
    ('normalize', Normalizer(norm='l2')),
    ('reduce', TruncatedSVD(n_components=50)),
    ('cluster', AgglomerativeClustering(n_clusters=30, affinity='euclidean', connectivity=False)),
])
pipe_full

Pipeline(steps=[('normalize', Normalizer()),
                ('reduce', TruncatedSVD(n_components=50)),
                ('cluster',
                 AgglomerativeClustering(connectivity=False, n_clusters=30))])

In [25]:
pipe_full.steps[-1]

('cluster', AgglomerativeClustering(connectivity=False, n_clusters=30))

In [26]:
type(pipe_full.steps[-1][1]).__name__

'AgglomerativeClustering'

In [49]:
# we can't remove by name
pipe_reduce = False
pipe_normalize = False

if not pipe_reduce:
    try:
        pipe_full.steps.remove('reduce')
    except Exception as e:
        print(f'reduce error: {e}')
        
if not pipe_normalize:
    try:
        pipe_full.steps.remove('normalize')
    except Exception as e:
        print(f'normalize error: {e}')

reduce error: list.remove(x): x not in list
normalize error: list.remove(x): x not in list


Instead, it looks like the safest way to remove a step is to iterate and remove only if we find the expected name.

Note that for each step we want to remove we need a separate for-loop - because the index will change when we remove a step.

In [51]:
d_pipe_steps_check = {
    'reduce': pipe_reduce,
    'normalize': pipe_normalize,
}

for name_, keep_ in d_pipe_steps_check.items():
    if not keep_:
        for i, tup_ in enumerate(pipe_full.steps):
            if tup_[0] == name_:
                print(f'removing {name_}')
                pipe_full.steps.pop(i)

pipe_full

removing reduce
removing normalize


Pipeline(steps=[('cluster',
                 AgglomerativeClustering(connectivity=False, n_clusters=30))])

In [52]:
# test inserting a step back into pipeline:
pipe_full.steps.insert(0, ('reduce', TruncatedSVD(n_components=50)))
pipe_full

Pipeline(steps=[('reduce', TruncatedSVD(n_components=50)),
                ('cluster',
                 AgglomerativeClustering(connectivity=False, n_clusters=30))])

In [53]:
# iterate over steps (so we can get the name for each transformer)
for tup_ in pipe_full.steps:
    print(tup_)

('reduce', TruncatedSVD(n_components=50))
('cluster', AgglomerativeClustering(connectivity=False, n_clusters=30))


# Test saving yaml

In [31]:
cfg_cluster_test_v040.config_dict

{'data_text_and_metadata': {'dataset_name': 'v0.4.0 inputs - Top Subreddits (no Geo) + Geo-relevant subs, comments: TBD',
  'bucket_name': 'i18n-subreddit-clustering',
  'folder_subreddits_text_and_meta': 'subreddits/top/2021-09-24',
  'folder_posts_text_and_meta': 'posts/top/2021-09-27',
  'folder_comments_text_and_meta': 'comments/top/2021-10-04',
  'comments_vectorized_mlflow_uuids': ['5f10cd75334142168a6ebb787e477c1f',
   '2fcfefc3d5af43328168d3478b4fdeb6'],
  'comments_vectorized_gcs': ['gs://i18n-subreddit-clustering/mlflow/mlruns/14/5f10cd75334142168a6ebb787e477c1f/artifacts/df_vect_comments/*.parquet',
   'gs://i18n-subreddit-clustering/mlflow/mlruns/14/2fcfefc3d5af43328168d3478b4fdeb6/artifacts/df_vect_comments/*.parquet'],
  'comments_vectorized_mlflow_uuids_lowercase': None,
  'posts_vectorized_mlflow_uuids': ['8eef951842a34a6e81d176b15ae74afd'],
  'posts_vectorized_mlflow_uuids_lowercase': ['537514ab3c724b10903000501802de0e'],
  'subreddit_meta_vectorized_mlflow_uuids': ['8

In [32]:
import yaml

with open('djb_test.yaml', 'w') as f:
    yaml.dump(cfg_cluster_test_v040.config_dict, f)

In [33]:
with open('djb_test.yaml', 'w') as f:
    yaml.dump(cfg_cluster_test_v040.config_dict, f, default_flow_style=False, allow_unicode=True)